In [1]:
import gym
import cv2
import numpy as np
import pandas as pd

In [2]:
env = gym.make('Breakout-ram-v0')

In [5]:
cols = ['Obs'+str(i) for i in range(128)]
cols.extend(['episode', 'action', 'reward'])

episodes = 100
discount_factor = 0.5
memory_df = pd.DataFrame(columns=cols)

for i_episode in range(episodes):
    done = False
    obs = env.reset()
    ep_reward, prev_reward = 0, 0
    for i in range(10):
        action = env.action_space.sample()
        prev_obs, _, _, _ = env.step(action)
    
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        ep_reward = (reward - prev_reward)*10 + ep_reward*discount_factor - .1
        memory_df.loc[len(memory_df.index)] = np.hstack((prev_obs, [i_episode, action, ep_reward]))
        prev_reward = reward
        prev_obs = obs

In [6]:
memory_df.to_csv('random_data_ram.csv', index=False)

In [6]:
data = pd.read_csv('random_data_ram1.csv')

In [7]:
data.groupby(by='episode')['hits'].max().sum()

111.0

## Epsilon Greedy

In [1]:
from tensorflow.keras.models import load_model
from helper_funcs import model_input
import pandas as pd
import numpy as np
import gym

In [2]:
randomness = False
path = 'C:/Users/INTERN/Downloads/VISHESH/DATA/'
try:
    if not randomness:
        model = load_model('model1_5')
        print('Model Loaded')
except:
    randomness = True

env = gym.make('Breakout-ram-v0')
np.random.seed(0)
env.seed(0)

Model Loaded


(2968811710, 3677149159)

In [3]:
cols = ['Obs'+str(i) for i in range(128)]
cols.extend(['episode', 'action', 'reward', 'hits'])

episodes = 100
discount_factor = 0.9
epsilon = 0
memory_df = pd.DataFrame(columns=cols)

for i_episode in range(episodes):
    done = False
    obs = env.reset()
    ep_reward, prev_reward, hits = 0, 0, 0
    for i in range(10):
        action = env.action_space.sample()
        prev_obs, _, _, _ = env.step(action)
    
    while not done:
        if np.random.rand() < epsilon or randomness:
            action = env.action_space.sample()
        else:
            x = model.predict(model_input(prev_obs, 4))
            action = np.argmax(x)
            # break
        obs, reward, done, info = env.step(action)
        hits += reward
        ep_reward = reward*10 + ep_reward*discount_factor - .1
        memory_df.loc[len(memory_df.index)] = np.hstack((prev_obs, [i_episode, action, ep_reward, hits]))
        prev_reward = reward
        prev_obs = obs
    print(i_episode, hits)

KeyboardInterrupt: 

In [32]:
memory_df.to_csv('random_data_ram6.csv', index=False)
print(memory_df.shape)

(24948, 132)


In [ ]:
memory_df.groupby(by='episode')['hits'].max().sum()

130.0